Trabajo con Datos con el Core: Selección de Filas
====

* Última modificación: 12 de septiembre de 2022.

Preparación
---

In [1]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [2]:
from sqlalchemy import Column, Integer, String, Table

user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

In [3]:
from sqlalchemy import ForeignKey

address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),  # Clave foranea
    Column("email_address", String, nullable=False),
)

In [4]:
from sqlalchemy import create_engine

engine = create_engine(
    "sqlite+pysqlite:///:memory:",
    echo=True,
    future=True,
)

metadata_obj.create_all(engine)

2022-09-13 22:00:33,919 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 22:00:33,919 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-09-13 22:00:33,920 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-13 22:00:33,921 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-09-13 22:00:33,921 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-13 22:00:33,922 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-09-13 22:00:33,922 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-13 22:00:33,923 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-09-13 22:00:33,924 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-13 22:00:33,925 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-09-13 22:00:33,926 INFO sqlalchemy.engine.Engine [no key 0.00055s] ()
2022-09-13 22:00:33,927 INFO sqlalchemy.engine.Engine 
C

In [5]:
from sqlalchemy import insert

with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "spongebob", "fullname": "Spongebob Squarepants"},
            {"name": "sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"},
        ],
    )
    conn.commit()

2022-09-13 22:00:34,008 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 22:00:34,009 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-09-13 22:00:34,010 INFO sqlalchemy.engine.Engine [generated in 0.00181s] (('spongebob', 'Spongebob Squarepants'), ('sandy', 'Sandy Cheeks'), ('patrick', 'Patrick Star'))
2022-09-13 22:00:34,011 INFO sqlalchemy.engine.Engine COMMIT


SELECT ...
---

In [6]:
from sqlalchemy import select

stmt = select(user_table).where(user_table.c.name == "spongebob")
print(stmt)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


In [7]:
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

2022-09-13 22:00:34,145 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 22:00:34,146 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-09-13 22:00:34,146 INFO sqlalchemy.engine.Engine [generated in 0.00188s] ('spongebob',)
(1, 'spongebob', 'Spongebob Squarepants')
2022-09-13 22:00:34,148 INFO sqlalchemy.engine.Engine ROLLBACK


COLUMNS y FROM
----

In [8]:
print(select(user_table))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


In [9]:
print(select(user_table.c.name, user_table.c.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


Expresiones SQL etiquetadas
---

In [10]:
from sqlalchemy import cast, func

stmt = select(
    ("Username: " + user_table.c.name).label("username"),
).order_by(user_table.c.name)

with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.username}")

2022-09-13 22:00:34,496 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 22:00:34,497 INFO sqlalchemy.engine.Engine SELECT ? || user_account.name AS username 
FROM user_account ORDER BY user_account.name
2022-09-13 22:00:34,497 INFO sqlalchemy.engine.Engine [generated in 0.00161s] ('Username: ',)
Username: patrick
Username: sandy
Username: spongebob
2022-09-13 22:00:34,498 INFO sqlalchemy.engine.Engine ROLLBACK


Expresiones con columnas textuales
---

In [11]:
from sqlalchemy import text

stmt = select(text("'some phrase'"), user_table.c.name).order_by(user_table.c.name)

with engine.connect() as conn:
    print(conn.execute(stmt).all())

2022-09-13 22:00:34,603 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 22:00:34,605 INFO sqlalchemy.engine.Engine SELECT 'some phrase', user_account.name 
FROM user_account ORDER BY user_account.name
2022-09-13 22:00:34,605 INFO sqlalchemy.engine.Engine [generated in 0.00172s] ()
[('some phrase', 'patrick'), ('some phrase', 'sandy'), ('some phrase', 'spongebob')]
2022-09-13 22:00:34,606 INFO sqlalchemy.engine.Engine ROLLBACK


In [12]:
from sqlalchemy import literal_column

stmt = select(literal_column("'some phrase'").label("p"), user_table.c.name).order_by(
    user_table.c.name
)

with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.p}, {row.name}")

2022-09-13 22:00:34,716 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 22:00:34,717 INFO sqlalchemy.engine.Engine SELECT 'some phrase' AS p, user_account.name 
FROM user_account ORDER BY user_account.name
2022-09-13 22:00:34,717 INFO sqlalchemy.engine.Engine [generated in 0.00155s] ()
some phrase, patrick
some phrase, sandy
some phrase, spongebob
2022-09-13 22:00:34,718 INFO sqlalchemy.engine.Engine ROLLBACK


WHERE
---

In [13]:
print(user_table.c.name == "squidward")

user_account.name = :name_1


In [14]:
print(address_table.c.user_id > 10)

address.user_id > :user_id_1


In [15]:
print(select(user_table).where(user_table.c.name == "squidward"))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


In [16]:
print(
    select(address_table.c.email_address)
    .where(user_table.c.name == "squidward")
    .where(address_table.c.user_id == user_table.c.id)
)

SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id


In [17]:
print(
    select(address_table.c.email_address).where(
        user_table.c.name == "squidward", address_table.c.user_id == user_table.c.id
    )
)

SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id


FROM ... JOIN
---

In [18]:
print(select(user_table.c.name))

SELECT user_account.name 
FROM user_account


In [19]:
print(
    select(user_table.c.name, address_table.c.email_address).join_from(
        user_table, address_table
    )
)

SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


In [20]:
print(select(user_table.c.name, address_table.c.email_address).join(address_table))

SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


In [21]:
print(select(address_table.c.email_address).select_from(user_table).join(address_table))

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


In [22]:
from sqlalchemy import func

print(select(func.count("*")).select_from(user_table))

SELECT count(:count_2) AS count_1 
FROM user_account


ON
---

In [23]:
print(
    select(address_table.c.email_address)
    .select_from(user_table)
    .join(address_table, user_table.c.id == address_table.c.user_id)
)

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


OUTER y FULL joins

In [24]:
print(select(user_table).join(address_table, isouter=True))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account LEFT OUTER JOIN address ON user_account.id = address.user_id


In [25]:
print(select(user_table).join(address_table, full=True))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account FULL OUTER JOIN address ON user_account.id = address.user_id


ORDER BY
---

In [26]:
print(select(user_table).order_by(user_table.c.name))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account ORDER BY user_account.name


ALIAS
---

In [27]:
user_alias_1 = user_table.alias()
user_alias_2 = user_table.alias()
print(
    select(user_alias_1.c.name, user_alias_2.c.name).join_from(
        user_alias_1, user_alias_2, user_alias_1.c.id > user_alias_2.c.id
    )
)

SELECT user_account_1.name, user_account_2.name AS name_1 
FROM user_account AS user_account_1 JOIN user_account AS user_account_2 ON user_account_1.id > user_account_2.id


Subconsultas
---

In [28]:
subq = (
    select(func.count(address_table.c.id).label("count"), address_table.c.user_id)
    .group_by(address_table.c.user_id)
    .subquery()
)

print(subq)

SELECT count(address.id) AS count, address.user_id 
FROM address GROUP BY address.user_id


In [29]:
print(select(subq.c.user_id, subq.c.count))

SELECT anon_1.user_id, anon_1.count 
FROM (SELECT count(address.id) AS count, address.user_id AS user_id 
FROM address GROUP BY address.user_id) AS anon_1


In [30]:
stmt = select(user_table.c.name, user_table.c.fullname, subq.c.count).join_from(
    user_table, subq
)

print(stmt)

SELECT user_account.name, user_account.fullname, anon_1.count 
FROM user_account JOIN (SELECT count(address.id) AS count, address.user_id AS user_id 
FROM address GROUP BY address.user_id) AS anon_1 ON user_account.id = anon_1.user_id


In [31]:
subq = (
    select(func.count(address_table.c.id).label("count"), address_table.c.user_id)
    .group_by(address_table.c.user_id)
    .cte()
)

stmt = select(user_table.c.name, user_table.c.fullname, subq.c.count).join_from(
    user_table, subq
)

print(stmt)

WITH anon_1 AS 
(SELECT count(address.id) AS count, address.user_id AS user_id 
FROM address GROUP BY address.user_id)
 SELECT user_account.name, user_account.fullname, anon_1.count 
FROM user_account JOIN anon_1 ON user_account.id = anon_1.user_id


Escalares y subconsultas correlacionadas
---

In [32]:
subq = (
    select(func.count(address_table.c.id))
    .where(user_table.c.id == address_table.c.user_id)
    .scalar_subquery()
)

print(subq)

(SELECT count(address.id) AS count_1 
FROM address, user_account 
WHERE user_account.id = address.user_id)


In [33]:
print(subq == 5)

(SELECT count(address.id) AS count_1 
FROM address, user_account 
WHERE user_account.id = address.user_id) = :param_1


In [34]:
stmt = select(user_table.c.name, subq.label("address_count"))
print(stmt)

SELECT user_account.name, (SELECT count(address.id) AS count_1 
FROM address 
WHERE user_account.id = address.user_id) AS address_count 
FROM user_account


In [35]:
subq = (
    select(func.count(address_table.c.id))
    .where(user_table.c.id == address_table.c.user_id)
    .scalar_subquery()
    .correlate(user_table)
)

In [36]:
with engine.connect() as conn:

    result = conn.execute(
        select(
            user_table.c.name,
            address_table.c.email_address,
            subq.label("address_count"),
        )
        .join_from(user_table, address_table)
        .order_by(user_table.c.id, address_table.c.id)
    )

    print(result.all())

2022-09-13 22:00:39,797 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 22:00:39,798 INFO sqlalchemy.engine.Engine SELECT user_account.name, address.email_address, (SELECT count(address.id) AS count_1 
FROM address 
WHERE user_account.id = address.user_id) AS address_count 
FROM user_account JOIN address ON user_account.id = address.user_id ORDER BY user_account.id, address.id
2022-09-13 22:00:39,799 INFO sqlalchemy.engine.Engine [generated in 0.00184s] ()
[]
2022-09-13 22:00:39,800 INFO sqlalchemy.engine.Engine ROLLBACK


LATERAL
---

In [37]:
subq = (
    select(
        func.count(address_table.c.id).label("address_count"),
        address_table.c.email_address,
        address_table.c.user_id,
    )
    .where(user_table.c.id == address_table.c.user_id)
    .lateral()
)

stmt = (
    select(user_table.c.name, subq.c.address_count, subq.c.email_address)
    .join_from(user_table, subq)
    .order_by(user_table.c.id, subq.c.email_address)
)

print(stmt)

SELECT user_account.name, anon_1.address_count, anon_1.email_address 
FROM user_account JOIN LATERAL (SELECT count(address.id) AS address_count, address.email_address AS email_address, address.user_id AS user_id 
FROM address 
WHERE user_account.id = address.user_id) AS anon_1 ON user_account.id = anon_1.user_id ORDER BY user_account.id, anon_1.email_address


UNION, UNION ALL y otras operaciones sobre conjuntos
---

In [38]:
from sqlalchemy import union_all

stmt1 = select(user_table).where(user_table.c.name == "sandy")
stmt2 = select(user_table).where(user_table.c.name == "spongebob")

u = union_all(stmt1, stmt2)

with engine.connect() as conn:
    result = conn.execute(u)
    print(result.all())

2022-09-13 22:00:40,086 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 22:00:40,087 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ? UNION ALL SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-09-13 22:00:40,088 INFO sqlalchemy.engine.Engine [generated in 0.00160s] ('sandy', 'spongebob')
[(2, 'sandy', 'Sandy Cheeks'), (1, 'spongebob', 'Spongebob Squarepants')]
2022-09-13 22:00:40,089 INFO sqlalchemy.engine.Engine ROLLBACK


In [39]:
u_subq = u.subquery()

stmt = (
    select(u_subq.c.name, address_table.c.email_address)
    .join_from(address_table, u_subq)
    .order_by(u_subq.c.name, address_table.c.email_address)
)

with engine.connect() as conn:
    result = conn.execute(stmt)
    print(result.all())

2022-09-13 22:00:40,159 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 22:00:40,160 INFO sqlalchemy.engine.Engine SELECT anon_1.name, address.email_address 
FROM address JOIN (SELECT user_account.id AS id, user_account.name AS name, user_account.fullname AS fullname 
FROM user_account 
WHERE user_account.name = ? UNION ALL SELECT user_account.id AS id, user_account.name AS name, user_account.fullname AS fullname 
FROM user_account 
WHERE user_account.name = ?) AS anon_1 ON anon_1.id = address.user_id ORDER BY anon_1.name, address.email_address
2022-09-13 22:00:40,161 INFO sqlalchemy.engine.Engine [generated in 0.00170s] ('sandy', 'spongebob')
[]
2022-09-13 22:00:40,162 INFO sqlalchemy.engine.Engine ROLLBACK


EXISTS
---

In [40]:
subq = (
    select(func.count(address_table.c.id))
    .where(user_table.c.id == address_table.c.user_id)
    .group_by(address_table.c.user_id)
    .having(func.count(address_table.c.id) > 1)
).exists()

with engine.connect() as conn:
    result = conn.execute(select(user_table.c.name).where(subq))
    print(result.all())

2022-09-13 22:00:40,247 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 22:00:40,248 INFO sqlalchemy.engine.Engine SELECT user_account.name 
FROM user_account 
WHERE EXISTS (SELECT count(address.id) AS count_1 
FROM address 
WHERE user_account.id = address.user_id GROUP BY address.user_id 
HAVING count(address.id) > ?)
2022-09-13 22:00:40,249 INFO sqlalchemy.engine.Engine [generated in 0.00175s] (1,)
[]
2022-09-13 22:00:40,250 INFO sqlalchemy.engine.Engine ROLLBACK


In [41]:
subq = (
    select(address_table.c.id).where(user_table.c.id == address_table.c.user_id)
).exists()

with engine.connect() as conn:
    result = conn.execute(select(user_table.c.name).where(~subq))
    print(result.all())

2022-09-13 22:00:40,309 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 22:00:40,311 INFO sqlalchemy.engine.Engine SELECT user_account.name 
FROM user_account 
WHERE NOT (EXISTS (SELECT address.id 
FROM address 
WHERE user_account.id = address.user_id))
2022-09-13 22:00:40,311 INFO sqlalchemy.engine.Engine [generated in 0.00175s] ()
[('spongebob',), ('sandy',), ('patrick',)]
2022-09-13 22:00:40,312 INFO sqlalchemy.engine.Engine ROLLBACK


Funciones SQL
---

In [42]:
print(select(func.count()).select_from(user_table))

SELECT count(*) AS count_1 
FROM user_account


In [43]:
print(select(func.lower("A String With Much UPPERCASE")))

SELECT lower(:lower_2) AS lower_1


In [44]:
stmt = select(func.now())

with engine.connect() as conn:
    result = conn.execute(stmt)
    print(result.all())

2022-09-13 22:00:40,527 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 22:00:40,528 INFO sqlalchemy.engine.Engine SELECT CURRENT_TIMESTAMP AS now_1
2022-09-13 22:00:40,529 INFO sqlalchemy.engine.Engine [generated in 0.00178s] ()
[(datetime.datetime(2022, 9, 14, 3, 0, 40),)]
2022-09-13 22:00:40,530 INFO sqlalchemy.engine.Engine ROLLBACK


In [45]:
print(select(func.some_crazy_function(user_table.c.name, 17)))

SELECT some_crazy_function(user_account.name, :some_crazy_function_2) AS some_crazy_function_1 
FROM user_account


In [46]:
from sqlalchemy.dialects import postgresql

print(select(func.now()).compile(dialect=postgresql.dialect()))

SELECT now() AS now_1


In [47]:
from sqlalchemy.dialects import oracle

print(select(func.now()).compile(dialect=oracle.dialect()))

SELECT CURRENT_TIMESTAMP AS now_1 FROM DUAL


Retorno de tipos
---

In [48]:
func.now().type

DateTime()

In [49]:
from sqlalchemy import JSON

function_expr = func.json_object('{a, 1, b, "def", c, 3.5}', type_=JSON)

In [50]:
stmt = select(function_expr["def"])
print(stmt)

SELECT json_object(:json_object_1)[:json_object_2] AS anon_1


Funciones Built-in
---

In [51]:
m1 = func.max(Column("some_int", Integer))
m1.type

Integer()

In [52]:
m2 = func.max(Column("some_str", String))
m2.type

String()

In [53]:
func.now().type

DateTime()

In [54]:
func.current_date().type

Date()

In [55]:
func.now().type

DateTime()

In [56]:
func.current_date().type

Date()

In [57]:
func.concat("x", "y").type

String()

In [58]:
func.concat("x", "y").type

String()

In [59]:
func.upper("lowercase").type

NullType()

In [60]:
print(select(func.upper("lowercase") + " suffix"))

SELECT upper(:upper_1) || :upper_2 AS anon_1


In [61]:
stmt = (
    select(
        func.row_number().over(partition_by=user_table.c.name),
        user_table.c.name,
        address_table.c.email_address,
    )
    .select_from(user_table)
    .join(address_table)
)

with engine.connect() as conn:
    result = conn.execute(stmt)
    print(result.all())

2022-09-13 22:00:43,107 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 22:00:43,109 INFO sqlalchemy.engine.Engine SELECT row_number() OVER (PARTITION BY user_account.name) AS anon_1, user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id
2022-09-13 22:00:43,109 INFO sqlalchemy.engine.Engine [generated in 0.00185s] ()
[]
2022-09-13 22:00:43,110 INFO sqlalchemy.engine.Engine ROLLBACK


In [62]:
stmt = (
    select(
        func.count().over(order_by=user_table.c.name),
        user_table.c.name,
        address_table.c.email_address,
    )
    .select_from(user_table)
    .join(address_table)
)

with engine.connect() as conn:
    result = conn.execute(stmt)
    print(result.all())

2022-09-13 22:00:43,206 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 22:00:43,207 INFO sqlalchemy.engine.Engine SELECT count(*) OVER (ORDER BY user_account.name) AS anon_1, user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id
2022-09-13 22:00:43,208 INFO sqlalchemy.engine.Engine [generated in 0.00187s] ()
[]
2022-09-13 22:00:43,209 INFO sqlalchemy.engine.Engine ROLLBACK


In [63]:
print(
    func.unnest(
        func.percentile_disc([0.25, 0.5, 0.75, 1]).within_group(user_table.c.name)
    )
)

unnest(percentile_disc(:percentile_disc_1) WITHIN GROUP (ORDER BY user_account.name))


In [64]:
stmt = (
    select(
        func.count(address_table.c.email_address).filter(user_table.c.name == "sandy"),
        func.count(address_table.c.email_address).filter(
            user_table.c.name == "spongebob"
        ),
    )
    .select_from(user_table)
    .join(address_table)
)

with engine.connect() as conn:
    result = conn.execute(stmt)
    print(result.all())

2022-09-13 22:00:43,388 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 22:00:43,389 INFO sqlalchemy.engine.Engine SELECT count(address.email_address) FILTER (WHERE user_account.name = ?) AS anon_1, count(address.email_address) FILTER (WHERE user_account.name = ?) AS anon_2 
FROM user_account JOIN address ON user_account.id = address.user_id
2022-09-13 22:00:43,390 INFO sqlalchemy.engine.Engine [generated in 0.00178s] ('sandy', 'spongebob')
[(0, 0)]
2022-09-13 22:00:43,391 INFO sqlalchemy.engine.Engine ROLLBACK


Table-Valued Functions
---

In [65]:
onetwothree = func.json_each('["one", "two", "three"]').table_valued("value")

stmt = select(onetwothree).where(onetwothree.c.value.in_(["two", "three"]))

with engine.connect() as conn:
    result = conn.execute(stmt)
    result.all()

2022-09-13 22:00:43,471 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 22:00:43,472 INFO sqlalchemy.engine.Engine SELECT anon_1.value 
FROM json_each(?) AS anon_1 
WHERE anon_1.value IN (?, ?)
2022-09-13 22:00:43,472 INFO sqlalchemy.engine.Engine [generated in 0.00187s] ('["one", "two", "three"]', 'two', 'three')
2022-09-13 22:00:43,474 INFO sqlalchemy.engine.Engine ROLLBACK


Column Valued Functions
---

In [66]:
from sqlalchemy import func, select

stmt = select(func.json_array_elements('["one", "two"]').column_valued("x"))
print(stmt)

SELECT x 
FROM json_array_elements(:json_array_elements_1) AS x


In [67]:
from sqlalchemy.dialects import oracle

stmt = select(func.scalar_strings(5).column_valued("s"))
print(stmt.compile(dialect=oracle.dialect()))

SELECT COLUMN_VALUE s 
FROM TABLE (scalar_strings(:scalar_strings_1)) s


Conversión de tipos
--

In [68]:
from sqlalchemy import cast

stmt = select(cast(user_table.c.id, String))

with engine.connect() as conn:
    result = conn.execute(stmt)
    result.all()

2022-09-13 22:00:44,375 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 22:00:44,376 INFO sqlalchemy.engine.Engine SELECT CAST(user_account.id AS VARCHAR) AS id 
FROM user_account
2022-09-13 22:00:44,377 INFO sqlalchemy.engine.Engine [generated in 0.00188s] ()
2022-09-13 22:00:44,378 INFO sqlalchemy.engine.Engine ROLLBACK


In [69]:
from sqlalchemy import JSON

print(cast("{'a': 'b'}", JSON)["a"])

CAST(:param_1 AS JSON)[:param_2]


In [70]:
import json

from sqlalchemy import JSON, type_coerce
from sqlalchemy.dialects import mysql

s = select(type_coerce({"some_key": {"foo": "bar"}}, JSON)["some_key"])
print(s.compile(dialect=mysql.dialect()))

SELECT JSON_EXTRACT(%s, %s) AS anon_1
